# Predicting the daily average temperature

We will build a couple of simple POC models to predict the daily average temperature in California using data from the years 2019 and 2020. For the purpose of this study, we will just focus on univariate time series prediction.

A lot of business rely on weather data to make decisions. For example, insurance companies may use weather data to evaluate the probability and cost of insurance claims, or supermarkets may use weather to help decide what to stock.

## Data exploration

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error as mse
from neuralprophet import NeuralProphet

from weatherPredict.ts_funcs import stationary_test
from weatherPredict.plotting import ARIMA_predict_plot

In [2]:
df = pd.read_csv(os.path.join("data", "weather_data.csv")).drop(columns = ['Unnamed: 0'])
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data\\weather_data.csv'

In [ ]:
df.columns

We focus on the columns that we are interested in. For now, let's look at the temperature columns and precipitation.

In [ ]:
# process
cols_to_keep = [
    'datetime',
    'tempmax',
    'tempmin',
    'temp',
    'precip',
]

df = df[cols_to_keep]
df['datetime'] = pd.to_datetime(df.loc[:,'datetime'])

In [ ]:
# lets check for missing values
df.isnull().values.any()
df.isnull().sum()
# there are none so no need to fill entries

In [ ]:
# let's construct some smooth data so it's easier to visualise
df['temp_7day_MA'] = df['temp'].rolling(window = 7).mean()
df['tempmin_7day_MA'] = df['tempmin'].rolling(window = 7).mean()
df['tempmax_7day_MA'] = df['tempmax'].rolling(window = 7).mean()
df['precip_7day_sum'] = df['precip'].rolling(window = 7).sum()

In [ ]:
df.plot(x = 'datetime', y = ['temp', 'temp_7day_MA'])
plt.grid();
plt.ylim([0,35]);
plt.title('Daily average temperature in California');
plt.ylabel('Temperature (degrees Celsius)');

In [ ]:
plt.fill_between(df['datetime'], df['tempmin_7day_MA'], df['tempmax_7day_MA'], alpha = 0.3)
plt.title("Daily minimum and maximum temperature ranges in California");
plt.ylabel('Temperature (degrees Celsius)');

In [ ]:
df.plot('datetime', 'precip_7day_sum')
plt.title('Weekly total precipitation in California')
plt.ylabel('Weekly precipitation (mm)')

We see that there is unsurprisinly yearly seasoanality in temperature trends. The daily temperature range remains relatively constant as a proportion of average temperature, and so we will not attempt to predict either of those. Instead we will focus on the daily average tempperature.

Furthermore, it appears there is little regular rain in California. In other words, it rains infrequently. There does appear to be more rain around January than in other times of the year, but there isn't much of a trend. Therefore, it also does not seem appropriate to model precipitation using time series analysis techniques.

## Time Series Analysis

We will explore some of the properties of the time series data, and also explore how to transform the data so it is suitable for an ARIMA model.

In [ ]:
df.set_index('datetime', inplace = True)

We can use the statsmodels seasonal decompose function to decompose the time series into a trend and seasonal component, and what the leftover residual is.

Seasonal decompose is not picking up the annual seasonality. However, we would likely need more than two years of data to do this. As a result, the overall trend is still quite noisy.

In [ ]:
decomposed_data = seasonal_decompose(df['temp'])
fig = decomposed_data.plot();
fig.set_size_inches((16, 9))

ARIMA models require the data to be stationary. Therefore, we perform the Augmented Dicker-Fuller test to test for stationarity.

In [ ]:
# evaluate stationarity
stationary_test(df, 'temp')

Our threshold p-value is 0.05. As the p-value is greater than 0.05, we fail to reject the null hypothesis that the time-series is non-stationary.

Taking the log of a time series analysis is one way to reduce the deviation from the mean

In [ ]:
# data is not stationary
df['log_temp'] = np.log(df['temp'])

In [ ]:
df.plot(y = 'log_temp')

In [ ]:
# evaluate stationarity
stationary_test(df, 'log_temp')

It is still non-stationary.

The d-order of an ARIMA model is the order of differencing used. We want to find the smallest number d such that our time series is stationary. Let's take the first order difference.

In [ ]:
df['diff_log_temp'] = df['log_temp'].diff()
stationary_test(df.dropna(), 'diff_log_temp')

Now our p-value < 0.05, so we can reject the null hypothesis, and infer that the time series is now stationary. We plot the difference of the log of the temp below so we can visually verify this.

In [ ]:
# now it is stationary implying a differencing term of 1 is appropriate
df['diff_log_temp'].plot()

Orders p and q of the ARIMA model can be informed by the autocorrelation and partial autocorrelation figures.

In [ ]:
plot_acf(df['diff_log_temp'].dropna());
plot_pacf(df['diff_log_temp'].dropna());

The above implies p and q values of 2, as the first significant value is with lags of 2. We therefore, as a first past, attempt to fit an ARIMA(2,2,1) model.

## Fitting ARIMA models

In [ ]:
model = ARIMA(df['log_temp'].values, order=(2,2,1))
model_fit = model.fit()
print(model_fit.summary())

Let's see how well the models fits to the data

In [ ]:
df['log_temp_predict'] = model_fit.predict()

In [ ]:
df.plot(y = ['log_temp', 'log_temp_predict'])

It appears as though it is fitting OK. However, this isn't informative as it only tells us it fits to data is has seen but doesn't inform us of the models predictive ability. So we split the data set into a training and test set and fit the model to the training data.

In [ ]:
train_df = df.iloc[:650]
test_df = df.iloc[650:]
model = ARIMA(train_df['log_temp'].values, order=(2,2,1))
model_fit = model.fit()

In [ ]:
ARIMA_predict_plot(model_fit, train_df, test_df, 'log_temp', model_type = 'ARIMA')

The model is clearly not very good, so let's investigate the accuracy of different ARIMA models. We investigate the following:

* There is seasonality in the data, however a simple ARIMA model does not model seasonalitiy.
* The ARIMA model orders may not be optimal.

Furthermore, there are additional seasonal orders in the SARIMA model, that can be difficult to choose by judgement. Therefore, we using the auto_arima function provided by the pmdarima package that attemptys to identify the optimal SARIMA parameters.

In [ ]:
arima_model = auto_arima(train_df['log_temp'])

In [ ]:
arima_model.summary()

In [ ]:
ARIMA_predict_plot(arima_model, train_df, test_df, 'log_temp', model_type = 'auto_arima')

## Exploring the Neural Prophet model.

The ARIMA and SARIMA models did not perform well on our of sample data. So let's try one more different class of model. We use the neural prophet package. This is a development of the fbprophet package.

In [ ]:
train_prophet_df = train_df[['log_temp']].reset_index().rename(columns = {'datetime' : 'ds', 'log_temp': 'y'})

In [ ]:
#basic model
n1 = NeuralProphet()
model = n1.fit(train_prophet_df, freq='D')

In [ ]:
prophet_df = df[['log_temp']].reset_index().rename(columns = {'datetime' : 'ds', 'log_temp': 'y'})
forecast = n1.predict(prophet_df)
forecast.tail()
plot = n1.plot(forecast)

The default model is not very good at fitting out of sample. Let's specify the model with things that we know. We know

* There is yearly seasonality
* There is not daily seasonality
* There is not weekly seasonality
* Over these two years, there is no growth (would be different if we had more data or were projecting over a longer period of time)

In [ ]:
# It is not very good, so let's further specify the model 
#basic model
n2 = NeuralProphet(yearly_seasonality = True, daily_seasonality = False, weekly_seasonality = False, growth = 'off')
model = n2.fit(train_prophet_df, freq='D')
prophet_df = df[['log_temp']].reset_index().rename(columns = {'datetime' : 'ds', 'log_temp': 'y'})
forecast = n2.predict(prophet_df)
forecast.tail()
plot = n2.plot(forecast)

# Model performance evaluation

In [ ]:
# models are auto_arima, n1 and n2
train_results_df = train_df[['log_temp']]
train_results_df['arima_predict'] = arima_model.predict_in_sample()
train_results_df['simple_NP'] = n1.predict(train_prophet_df)['yhat1'].values
train_results_df['specified_NP'] = n2.predict(train_prophet_df)['yhat1'].values

test_results_df = test_df[['log_temp']]
test_prophet_df = test_df[['log_temp']].reset_index().rename(columns = {'datetime' : 'ds', 'log_temp': 'y'})
test_results_df['arima_predict'] = arima_model.predict(len(test_df))
test_results_df['simple_NP'] = n1.predict(test_prophet_df)['yhat1'].values
test_results_df['specified_NP'] = n2.predict(test_prophet_df)['yhat1'].values

In [ ]:
print(mse(train_results_df['log_temp'], train_results_df['arima_predict']))
print(mse(train_results_df['log_temp'], train_results_df['simple_NP']))
print(mse(train_results_df['log_temp'], train_results_df['specified_NP']))

In [ ]:
print(mse(test_results_df['log_temp'], test_results_df['arima_predict']))
print(mse(test_results_df['log_temp'], test_results_df['simple_NP']))
print(mse(test_results_df['log_temp'], test_results_df['specified_NP']))

While the ARIMA model performs better than the neural prophet model on the training set, it doesn't predict out of sample values well.
The correctly specified neural prophet model performs much better out of sample.
This emphasises the importances of cross-validation.

## Conclusions and suggested next steps for modelling

We have shown that the neuralprophet model can predict general trends in average daily temperature about 3 months in the future in California for this given dataset. However, there are is a lot more to be done to this model before it can be verified, or add business value.

### Cross - validation

Cross-validation should be used to verify the performance of the model. That is, the model should be trained on different sections of the data and then used to predict out-of-sample data to verify that the model does not happen to perform well on this given training and test set.

### Use of exogeneous variables

Time series prediction assumes that the variable of interest can be explained by its past values. In the case of daily everage temperatures, we know this to not be the case. Therefore, we could use other variables to help us explain temperature. For example, wind speed, precipitation, humidity, temperature in neighbouring areas, etc.

### More data

We only have data for two years. This is clearly not enough. For the purpose of this project, I only used 2 years of data due to query limits on visual crossing, but inferring yearly seasonality would be difficult with just 2 years of data, though the nerual prophet model did ok.

Furthermore, depending on the business, more spatially granular data may be required. These results are for California as a whole. However, a restaurant for example may want to know the weather at each given store. In this case use of a vector auto-regression (VAR) model may be more appropriate.

### Use of CO2 levels

If we had more data, we would start to see a net upwards trend in temperature. This is correlated with CO2 levels. We could use CO2 as a feature, but also we can use projections of CO2 emissions to help predict future temperature.

### Correlate with a variable of interest to the business

If the goal of the project is to predict sales of ice-cream, for example, then a history of ice-cream sales would be useful. The correlation of ice-cream sales and past daily average temperatures could be evaluated, and a VAR model could be fit to the two time series.

### Exploration of other model choices

We have only expored neural prophet and ARIMA-type models in the this exercise. There are other model choices that have proven to be useful in time series analysis questions, particularly once more exogeneous variables and multiple time series are added, such as LSTM and XGBoost.

## Implementation/productionisation concerns

### Observing data drift

It is not unlikely that daily average temperatures could drift outside of the previously observed ranges. This will need to be monitored, and models re-trained accordingly.

If, for-example, the neural prophet model is fitted as above, where there is assumed to be no growth, then, if average temperatures trends upwards, this model will not fit so well anymore. It would be useful to fit a curve to the trend in annual temperatures to assure that the rate of increase in temperatures is not accelarating or deccelerating outside of previously assumed bounds.

### Evaluating model performance

It is important to ensure that the error on the test set doesn't start to drift upwards. If it does, it may warrant re-training of the model.

### Outliers (extreme events)

Temperature data, as seen above, is quite noisy. The model did a good job at predicting the general trend in temperatures, however, any univariate model is unlikely to predict sudden cold or warm days. This may affect the usefulness of the model to the businesss, as the extreme events may be more likely to have a business impact (such as on insurance companies)


### Other things to add when productionising model

* Continuous Integraton (automated unit-tests)
* Continuous Deployment (automated package build/dockerisation)
* Logging